In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_validate


# 1. Defining the problem statement
Look at each variable and do a philosophical analysis about their meaning and importance for this problem.

# 2. Collecting Data

In [2]:
dataset = pd.read_csv("train.csv")
df =dataset.copy()
test = pd.read_csv('test.csv')
sub_df = pd.read_csv('sample_submission.csv')

In [3]:
train_test = pd.concat([df,test])

In [4]:
# X_df = dataset.iloc[:, :-1]
# y_df = dataset.iloc[:,-1]

# X_train , X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=0.2, shuffle = True,random_state=121)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)


# 3. EDA(Exploratory Data Analysis) & Data Preprocessing


In [5]:
dataset.shape

(1460, 81)

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [7]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Missing_Value 

In [8]:
nan_dict = {"CNT":train_test.isna().sum(),
            "RATE": train_test.isna().sum()/train_test.shape[0] *100
}
nan_df = pd.DataFrame(nan_dict)
print(nan_df[nan_df["RATE"]>0].sort_values("CNT", ascending=False))

nan_df[nan_df["RATE"]>0].shape

               CNT       RATE
PoolQC        2909  99.657417
MiscFeature   2814  96.402878
Alley         2721  93.216855
Fence         2348  80.438506
SalePrice     1459  49.982871
FireplaceQu   1420  48.646797
LotFrontage    486  16.649538
GarageCond     159   5.447071
GarageYrBlt    159   5.447071
GarageFinish   159   5.447071
GarageQual     159   5.447071
GarageType     157   5.378554
BsmtExposure    82   2.809181
BsmtCond        82   2.809181
BsmtQual        81   2.774923
BsmtFinType2    80   2.740665
BsmtFinType1    79   2.706406
MasVnrType      24   0.822199
MasVnrArea      23   0.787941
MSZoning         4   0.137033
Functional       2   0.068517
BsmtHalfBath     2   0.068517
BsmtFullBath     2   0.068517
Utilities        2   0.068517
KitchenQual      1   0.034258
TotalBsmtSF      1   0.034258
BsmtUnfSF        1   0.034258
GarageCars       1   0.034258
GarageArea       1   0.034258
BsmtFinSF2       1   0.034258
BsmtFinSF1       1   0.034258
Exterior2nd      1   0.034258
Exterior1s

(35, 2)

### category variables which use NA as a type  
<pre>
Alley
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmyFinType2
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PoolQC
Fence
MiscFeature
</pre>



In [9]:
Not_NA=['Alley', 'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu', 'GarageType',
'GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
len(Not_NA)

14

#### Converting types: category varilables (obj --> numeric)

In [10]:
for i in Not_NA:
    print(i,train_test[i].unique())

Alley [nan 'Grvl' 'Pave']
BsmtQual ['Gd' 'TA' 'Ex' nan 'Fa']
BsmtCond ['TA' 'Gd' nan 'Fa' 'Po']
BsmtExposure ['No' 'Gd' 'Mn' 'Av' nan]
BsmtFinType1 ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
BsmtFinType2 ['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']
FireplaceQu [nan 'TA' 'Gd' 'Fa' 'Ex' 'Po']
GarageType ['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
GarageFinish ['RFn' 'Unf' 'Fin' nan]
GarageQual ['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
GarageCond ['TA' 'Fa' nan 'Gd' 'Po' 'Ex']
PoolQC [nan 'Ex' 'Fa' 'Gd']
Fence [nan 'MnPrv' 'GdWo' 'GdPrv' 'MnWw']
MiscFeature [nan 'Shed' 'Gar2' 'Othr' 'TenC']


In [11]:
change_dict = [{'Grvl':1,'Pave':2}, {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5},
{'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5},{'No':1 ,'Gd':2, 'Mn':3, 'Av':4},
{'GLQ':1, 'ALQ':2, 'Unf':3, 'Rec':4, 'BLQ':5, 'LwQ':6},
{'Unf':1, 'BLQ':2,'ALQ':3, 'Rec':4, 'LwQ':5, 'GLQ':6},
{'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5},
{'Attchd':1, 'Detchd':2, 'BuiltIn':3, 'CarPort':4,'Basment':5,'2Types':6},
{'RFn':1, 'Unf':2, 'Fin':3},
{'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5},{'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5},
{'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5},{'MnPrv':1, 'GdWo':2, 'GdPrv':3, 'MnWw':4},
{'Shed':1,'Gar2':2, 'Othr':3, 'TenC':4}]



In [12]:
for i in range(len(Not_NA)):
    train_test["T_"+Not_NA[i]] = train_test[Not_NA[i]].map(change_dict[i])
    train_test["T_"+Not_NA[i]].fillna(0, inplace= True)

In [13]:
for i in range(len(Not_NA)):
    print(train_test[train_test["T_"+Not_NA[i]]==0].shape[0]) # 0의 개수
    print(train_test[Not_NA[i]].isna().sum()) #원래 NA 개수
    print("-"*30)

2721
2721
------------------------------
81
81
------------------------------
82
82
------------------------------
82
82
------------------------------
79
79
------------------------------
80
80
------------------------------
1420
1420
------------------------------
157
157
------------------------------
159
159
------------------------------
159
159
------------------------------
159
159
------------------------------
2909
2909
------------------------------
2348
2348
------------------------------
2814
2814
------------------------------


In [14]:
train_test.drop(Not_NA, axis=1, inplace= True)

In [15]:
train_test.shape

(2919, 81)

In [16]:
#남은 결측치 확인
nan_dict = {"CNT":train_test.isna().sum(),
            "RATE": train_test.isna().sum()/train_test.shape[0] *100
}
nan_df = pd.DataFrame(nan_dict)
print(nan_df[nan_df["RATE"]>0].sort_values("CNT", ascending=False))

nan_df[nan_df["RATE"]>0].shape

# LotFrontage: Linear feet of street connected to property float
# GarageYrBlt: Year garage was built
# MasVnrType: Masonry veneer type 건물 외벽 object
# MasVnrArea: Masonry veneer area in square feet float
# Electrical: Electrical system object

               CNT       RATE
SalePrice     1459  49.982871
LotFrontage    486  16.649538
GarageYrBlt    159   5.447071
MasVnrType      24   0.822199
MasVnrArea      23   0.787941
MSZoning         4   0.137033
BsmtFullBath     2   0.068517
BsmtHalfBath     2   0.068517
Functional       2   0.068517
Utilities        2   0.068517
Exterior2nd      1   0.034258
BsmtFinSF1       1   0.034258
BsmtFinSF2       1   0.034258
BsmtUnfSF        1   0.034258
Exterior1st      1   0.034258
Electrical       1   0.034258
KitchenQual      1   0.034258
GarageCars       1   0.034258
GarageArea       1   0.034258
SaleType         1   0.034258
TotalBsmtSF      1   0.034258


(21, 2)

In [17]:
real_NA = list(nan_df[nan_df["RATE"]>0].index)

train_test['LotFrontage'] = train_test.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

real_NA.remove('LotFrontage')

for i in real_NA:
    train_test[i].fillna(train_test[i].mode()[0], inplace= True)

## Numeric

In [19]:
object_col= train_test.dtypes[train_test.dtypes == object].index

In [20]:
object_col

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')

In [21]:
# ##get dummies 인코딩
# df_copy = df.copy()
# for i in range(len(object_col)):
#     dummy_df = pd.get_dummies(df[object_col[i]])
#     df_copy = pd.concat([df_copy,dummy_df], axis =1)
#     # df = df.join(dummy_df)

# # dummy_df = pd.get_dummies(df[object_col[0]])
# # df_copy = pd.concat([df_copy,dummy_df], axis=1)
    

In [22]:
# Label 인코딩
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le_encoder = LabelEncoder()

for col in object_col:   
    train_test["LE_"+col] = le_encoder.fit_transform(train_test[col])    

In [23]:
#기존 컬럼 삭제
train_test.drop(object_col, axis = 1, inplace= True)

In [24]:
#남은 object type 있는지 확인
train_test.dtypes[train_test.dtypes == object].index

Index([], dtype='object')

## Scaling

In [29]:
y_df = dataset["SalePrice"]

In [30]:
#log transform skewed numeric features:
from scipy.stats import skew
numeric_feats = train_test.dtypes[train_test.dtypes != "object"].index
skewed_feats =  train_test[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats =skewed_feats.index

train_test[skewed_feats] = np.log1p(train_test[skewed_feats])
#y값 스케일링
y_df  = np.log1p(y_df)

In [31]:
#Id 제거
train_test.drop("Id", axis=1, inplace= True)

KeyError: "['Id'] not found in axis"

In [32]:
X_df = train_test.iloc[:1460,:]
test_df =train_test.iloc[1460:,:]

# 회귀분석

In [33]:
train_test.isna().sum()

MSSubClass          0
LotFrontage         0
LotArea             0
OverallQual         0
OverallCond         0
                   ..
LE_KitchenQual      0
LE_Functional       0
LE_PavedDrive       0
LE_SaleType         0
LE_SaleCondition    0
Length: 80, dtype: int64

## 우선 점수부터 보자

In [36]:
X_train , X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=0.2, shuffle = True,random_state=121)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

model1 = LinearRegression()
model1.fit(X_train,y_train)

pred = model1.predict(X_val)
mse = mean_squared_error(y_val, pred, squared = True) #MSE
rmse = mean_squared_error(y_val, pred, squared = False) #RMSE

print(f'MSE:{mse:.5f} RMSE {rmse:.5f}')

#dummy
#MSE:183766087910688064.00000 RMSE 428679469.89643 처음
#MSE:728536469.52603 RMSE 26991.41474 Id 제거
#MSE:969745875.68496 RMSE 31140.74302 scale 후에
#MSE:908127775.78686 RMSE 30135.15847 scale 후에

#-----------------------------------------------------------
#Label
# MSE:827344230.24246 RMSE 28763.59210 처음
# MSE:954940616.83012 RMSE 30902.11347 스케일 후
# MSE:1593369743.02272 RMSE 39917.03575 pca 후









(1168, 80) (292, 80) (1168,) (292,)
MSE:0.00006 RMSE 0.00745


In [49]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
lasso = Lasso(alpha = 1.0)
ridge = Ridge(alpha = 1.0)
elsnet = ElasticNet(alpha = 0.05, l1_ratio =0.1)

#RMSE mean, MSE mean, fold
models = [lasso, ridge, elsnet]
    # models = [LinearRegression()]
for model in models:
    score_list = cross_validate(model, X_df, y_df, 
                                 scoring = {'mse':'neg_mean_squared_error',
                                            'rmse':'neg_root_mean_squared_error'},
                                 cv= 10, return_train_score = False)
    mse_score = -score_list['test_mse']
    rmse_score = -score_list['test_rmse']
    print(f'{model.__class__.__name__} MSE: {mse_score.mean():.4f} RMSE: {rmse_score.mean():.4f}')

Lasso MSE: 0.0719 RMSE: 0.2674
Ridge MSE: 0.0147 RMSE: 0.1191
ElasticNet MSE: 0.0212 RMSE: 0.1441


In [56]:
# cv_model = GridSearchCV(ridge, param_grid = {'alpha':[0.5,1.0]},
#             scoring = "neg_root_mean_squared_error",
#             cv = 10)
# cv_model.fit(X_train, y_train)
# print(f'{cv_model.best_score_*-1:.5f}')
# print(cv_model.best_params_)

0.11901
{'alpha': 0.5}


In [58]:
cv_model = GridSearchCV(ridge, param_grid = {'alpha':[1.0,2.0]},
            scoring = "neg_root_mean_squared_error",
            cv = 10)
cv_model.fit(X_train, y_train)
print(f'{cv_model.best_score_*-1:.5f}')
print(cv_model.best_params_)

0.12956
{'alpha': 1.0}


In [59]:
## test 
test_pred = cv_model.predict(test_df)
pred_for_submit = np.expm1(test_pred)

In [60]:
pred_for_submit

array([117032.53850006, 151737.7225479 , 173480.52245195, ...,
       165873.25792143, 128130.42180567, 223718.84363169])

In [61]:
sub_df["SalePrice"] = pred_for_submit.reshape(-1,1)
sub_df.shape

(1459, 2)

In [62]:
sub_df.head()

,Id,SalePrice
0,1461,117032.538500
1,1462,151737.722548
2,1463,173480.522452
3,1464,188280.802994
4,1465,183277.578269


In [63]:
sub_df.to_csv("results.csv", index= False)